In [135]:
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
from PIL import Image
import numpy as np

#### https://gist.github.com/EncodeTS/6bbe8cb8bebad7a672f0d872561782d9

In [136]:
def vgg_face1(weights_path=None):
    img = Input(shape=(3, 224, 224))

    pad1_1 = ZeroPadding2D(padding=(1, 1))(img)
    conv1_1 = Convolution2D(64, 3, 3, activation='relu', name='conv1_1')(pad1_1)
    pad1_2 = ZeroPadding2D(padding=(1, 1))(conv1_1)
    conv1_2 = Convolution2D(64, 3, 3, activation='relu', name='conv1_2')(pad1_2)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2))(conv1_2)

    pad2_1 = ZeroPadding2D((1, 1))(pool1)
    conv2_1 = Convolution2D(128, 3, 3, activation='relu', name='conv2_1')(pad2_1)
    pad2_2 = ZeroPadding2D((1, 1))(conv2_1)
    conv2_2 = Convolution2D(128, 3, 3, activation='relu', name='conv2_2')(pad2_2)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2))(conv2_2)

    pad3_1 = ZeroPadding2D((1, 1))(pool2)
    conv3_1 = Convolution2D(256, 3, 3, activation='relu', name='conv3_1')(pad3_1)
    pad3_2 = ZeroPadding2D((1, 1))(conv3_1)
    conv3_2 = Convolution2D(256, 3, 3, activation='relu', name='conv3_2')(pad3_2)
    pad3_3 = ZeroPadding2D((1, 1))(conv3_2)
    conv3_3 = Convolution2D(256, 3, 3, activation='relu', name='conv3_3')(pad3_3)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2))(conv3_3)

    pad4_1 = ZeroPadding2D((1, 1))(pool3)
    conv4_1 = Convolution2D(512, 3, 3, activation='relu', name='conv4_1')(pad4_1)
    pad4_2 = ZeroPadding2D((1, 1))(conv4_1)
    conv4_2 = Convolution2D(512, 3, 3, activation='relu', name='conv4_2')(pad4_2)
    pad4_3 = ZeroPadding2D((1, 1))(conv4_2)
    conv4_3 = Convolution2D(512, 3, 3, activation='relu', name='conv4_3')(pad4_3)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4_3)

    pad5_1 = ZeroPadding2D((1, 1))(pool4)
    conv5_1 = Convolution2D(512, 3, 3, activation='relu', name='conv5_1')(pad5_1)
    pad5_2 = ZeroPadding2D((1, 1))(conv5_1)
    conv5_2 = Convolution2D(512, 3, 3, activation='relu', name='conv5_2')(pad5_2)
    pad5_3 = ZeroPadding2D((1, 1))(conv5_2)
    conv5_3 = Convolution2D(512, 3, 3, activation='relu', name='conv5_3')(pad5_3)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5_3)

    flat = Flatten()(pool5)
    fc6 = Dense(4096, activation='relu', name='fc6')(flat)
    fc6_drop = Dropout(0.5)(fc6)
    fc7 = Dense(4096, activation='relu', name='fc7')(fc6_drop)
    fc7_drop = Dropout(0.5)(fc7)
    out = Dense(2622, activation='softmax', name='fc8')(fc7_drop)

    model = Model(input=img, output=out)

    if weights_path:
        model.load_weights(weights_path)

    return model

In [137]:
def vgg_face2(weights_path=None):
    img = Input(shape=(3, 224, 224))

    pad1_1 = ZeroPadding2D(padding=(1, 1))(img)
    conv1_1 = Convolution2D(64, 3, 3, activation='relu', name='conv1_1')(pad1_1)
    pad1_2 = ZeroPadding2D(padding=(1, 1))(conv1_1)
    conv1_2 = Convolution2D(64, 3, 3, activation='relu', name='conv1_2')(pad1_2)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2))(conv1_2)

    pad2_1 = ZeroPadding2D((1, 1))(pool1)
    conv2_1 = Convolution2D(128, 3, 3, activation='relu', name='conv2_1')(pad2_1)
    pad2_2 = ZeroPadding2D((1, 1))(conv2_1)
    conv2_2 = Convolution2D(128, 3, 3, activation='relu', name='conv2_2')(pad2_2)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2))(conv2_2)

    pad3_1 = ZeroPadding2D((1, 1))(pool2)
    conv3_1 = Convolution2D(256, 3, 3, activation='relu', name='conv3_1')(pad3_1)
    pad3_2 = ZeroPadding2D((1, 1))(conv3_1)
    conv3_2 = Convolution2D(256, 3, 3, activation='relu', name='conv3_2')(pad3_2)
    pad3_3 = ZeroPadding2D((1, 1))(conv3_2)
    conv3_3 = Convolution2D(256, 3, 3, activation='relu', name='conv3_3')(pad3_3)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2))(conv3_3)

    pad4_1 = ZeroPadding2D((1, 1))(pool3)
    conv4_1 = Convolution2D(512, 3, 3, activation='relu', name='conv4_1')(pad4_1)
    pad4_2 = ZeroPadding2D((1, 1))(conv4_1)
    conv4_2 = Convolution2D(512, 3, 3, activation='relu', name='conv4_2')(pad4_2)
    pad4_3 = ZeroPadding2D((1, 1))(conv4_2)
    conv4_3 = Convolution2D(512, 3, 3, activation='relu', name='conv4_3')(pad4_3)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4_3)

    pad5_1 = ZeroPadding2D((1, 1))(pool4)
    conv5_1 = Convolution2D(512, 3, 3, activation='relu', name='conv5_1')(pad5_1)
    pad5_2 = ZeroPadding2D((1, 1))(conv5_1)
    conv5_2 = Convolution2D(512, 3, 3, activation='relu', name='conv5_2')(pad5_2)
    pad5_3 = ZeroPadding2D((1, 1))(conv5_2)
    conv5_3 = Convolution2D(512, 3, 3, activation='relu', name='conv5_3')(pad5_3)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5_3)

    fc6 = Convolution2D(4096, 7, 7, activation='relu', name='fc6')(pool5)
    fc6_drop = Dropout(0.5)(fc6)
    fc7 = Convolution2D(4096, 1, 1, activation='relu', name='fc7')(fc6_drop)
    fc7_drop = Dropout(0.5)(fc7)
    fc8 = Convolution2D(2622, 1, 1, name='fc8')(fc7_drop)
    flat = Flatten()(fc8)
    out = Activation('softmax')(flat)

    model = Model(input=img, output=out)

    if weights_path:
        model.load_weights(weights_path)

    return model

In [149]:
blankmodel = vgg_face2(None)

In [13]:
if False:
    im = Image.open('Colin_Firth.jpg')
    im = im.resize((224,224))
    im = np.array(im).astype(np.float32)
    if False:
        im[:,:,0] -= 129.1863
        im[:,:,1] -= 104.7624
        im[:,:,2] -= 93.5940
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)

    # Test pretrained model
    model = vgg_face('vgg-face-keras.h5')
    out = model.predict(im)
    print(out[0][0])

#### http://www.vlfeat.org/matconvnet/pretrained/#face-recognition
#### http://www.vlfeat.org/matconvnet/models/vgg-face.mat

In [14]:
from scipy.io import loadmat

In [16]:
data = loadmat('vgg-face.mat', matlab_compatible=False, struct_as_record=False)

In [17]:
type(data)

dict

In [104]:
data['meta'][0,0].classes[0,0].description

array([[array([u'A.J._Buckley'], 
      dtype='<U12')],
       [array([u'A.R._Rahman'], 
      dtype='<U11')],
       [array([u'Aamir_Khan'], 
      dtype='<U10')],
       ..., 
       [array([u'Zsa_Zsa_Gabor'], 
      dtype='<U13')],
       [array([u'Zulay_Henao'], 
      dtype='<U11')],
       [array([u'Zuleyka_Silver'], 
      dtype='<U14')]], dtype=object)

In [20]:
l = data['layers']

In [68]:
l.shape

(1, 37)

In [128]:
print(dir(l[0,10][0,0])), l[0,10][0,0].type[0], l[0,10][0,0].name[0]

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_fieldnames', 'dilate', 'learningRate', 'momentum', 'name', 'opts', 'pad', 'precious', 'stride', 'type', 'weightDecay', 'weights'] conv conv3_1


In [124]:
l[0,10][0,0].weights[0,0].shape, l[0,10][0,0].weights[0,1].shape

((3, 3, 128, 256), (256, 1))

In [154]:
blankmodel.layers[12].get_weights()[0].T.shape, blankmodel.layers[12].get_weights()[1].shape

((3, 3, 128, 256), (256,))

In [156]:
kerasnames = [lr.name for lr in blankmodel.layers]

for i in range(l.shape[1]):
    matname = l[0,i][0,0].name[0]
    if matname in kerasnames:
        kindex = kerasnames.index(matname)
        print matname
        print l[0,i][0,0].weights[0,0].shape, l[0,i][0,0].weights[0,1].shape
        print blankmodel.layers[kindex].get_weights()[0].T.shape, blankmodel.layers[kindex].get_weights()[1].shape
        print '------------------------------------------'

conv1_1
(3, 3, 3, 64) (64, 1)
(3, 3, 3, 64) (64,)
------------------------------------------
conv1_2
(3, 3, 64, 64) (64, 1)
(3, 3, 64, 64) (64,)
------------------------------------------
conv2_1
(3, 3, 64, 128) (128, 1)
(3, 3, 64, 128) (128,)
------------------------------------------
conv2_2
(3, 3, 128, 128) (128, 1)
(3, 3, 128, 128) (128,)
------------------------------------------
conv3_1
(3, 3, 128, 256) (256, 1)
(3, 3, 128, 256) (256,)
------------------------------------------
conv3_2
(3, 3, 256, 256) (256, 1)
(3, 3, 256, 256) (256,)
------------------------------------------
conv3_3
(3, 3, 256, 256) (256, 1)
(3, 3, 256, 256) (256,)
------------------------------------------
conv4_1
(3, 3, 256, 512) (512, 1)
(3, 3, 256, 512) (512,)
------------------------------------------
conv4_2
(3, 3, 512, 512) (512, 1)
(3, 3, 512, 512) (512,)
------------------------------------------
conv4_3
(3, 3, 512, 512) (512, 1)
(3, 3, 512, 512) (512,)
------------------------------------------
conv5_